In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=20, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)
# tuner.search_space_summary()
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Trial 247 Complete [01h 41m 32s]
val_weightedf2score: 0.44086945056915283

Best val_weightedf2score So Far: 0.6896040439605713
Total elapsed time: 5d 14h 17m 58s

Search: Running Trial #248

Value             |Best Value So Far |Hyperparameter
True              |True              |class_weight
2017_2018_2019    |2017_2018_2019    |training_years
l1                |l1l2              |kernel_regularizer
0.3               |0.5               |spatial_dropout
leaky_relu        |leaky_relu        |activation
2                 |4                 |pool_size
0.3               |0.1               |dropout
True              |False             |bias_initializer
0.01              |0.01              |learning_rate
binary_focal_cr...|binary_crossent...|loss_function
100               |34                |tuner/epochs
0                 |12                |tuner/initial_epoch
0                 |4                 |tuner/bracket
0                 |3                 |tuner/round

Epoch 1/100
3550/3550 ━━━━━

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"